In [1]:
DEVICES = "CPU" # Moguce vrijednosti: "CPU", "GPU:0", "GPU:1", "GPUS"
import os
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger('tensorflow').setLevel(logging.FATAL)

if DEVICES == 'CPU':
    os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
elif DEVICES == 'GPU:0':
    os.environ["CUDA_VISIBLE_DEVICES"] = '0'
elif DEVICES == 'GPU:1':
    os.environ["CUDA_VISIBLE_DEVICES"] = '1'


import tensorflow as tf

from dataset.dataset_loader import NerProcessor, FewNERDProcessor, create_tf_dataset_for_client, split_to_tf_datasets, batch_features
from utils.fl_utils import *
from models.model import build_BertNer, MaskedSparseCategoricalCrossentropy
from tokenization import FullTokenizer


from tqdm.notebook import tqdm
import json

import numpy as np

import nest_asyncio
nest_asyncio.apply()

import tensorflow_federated as tff

if DEVICES == 'CPU':
    cl_tf_devices = tf.config.list_logical_devices('CPU')
elif DEVICES == 'GPUS':
    cl_tf_devices = tf.config.list_logical_devices('GPU')
else:
    cl_tf_devices = tf.config.list_logical_devices('GPU')[:1]
tff.backends.native.set_local_execution_context(
    server_tf_device=tf.config.list_logical_devices('CPU')[0],
    client_tf_devices=cl_tf_devices)


# Pretrained models
TINY = 'uncased_L-2_H-128_A-2'
TINY_1 = 'uncased_L-4_H-128_A-2'
TINY_2 = 'uncased_L-6_H-128_A-2'
TINY_12_128='uncased_L-12_H-128_A-2'
TINY_8_128='uncased_L-8_H-128_A-2'
MINI = 'uncased_L-4_H-256_A-4'
SMALL = 'uncased_L-4_H-512_A-8'
MEDIUM = 'uncased_L-8_H-512_A-8'
BASE = 'uncased_L-12_H-768_A-12'


MODEL = os.path.join("models", TINY_12_128)
SEQ_LEN = 128
BATCH_SIZE = 32
PRETRAINED = False


processor = NerProcessor('dataset/conll')
datas="conll"
# processor = FewNERDProcessor('dataset/few_nerd')
# datas="few_nerd"
tokenizer = FullTokenizer(os.path.join(MODEL, "vocab.txt"), True)
train_features = processor.get_train_as_features(SEQ_LEN, tokenizer)
eval_features = processor.get_test_as_features(SEQ_LEN, tokenizer)

# Name of file
modelSize=""
pretr=""
# change dataset name in datas

modelSize = {
  'models/uncased_L-2_H-128_A-2': lambda: "tiny",
  'models/uncased_L-12_H-128_A-2': lambda: "tiny_12_128",
  'models/uncased_L-8_H-128_A-2': lambda: "tiny_8_128",
    'models/uncased_L-4_H-128_A-2': lambda: 'tiny_1',
    'models/uncased_L-6_H-128_A-2': lambda: 'tiny_2',
  'models/uncased_L-4_H-256_A-4': lambda: "mini",
  'models/uncased_L-4_H-512_A-8': lambda: "small",
  'models/uncased_L-8_H-512_A-8': lambda: "medium",
  'models/uncased_L-12_H-768_A-12': lambda: "base"
}[MODEL]()

if PRETRAINED == True:
    pretr="pretrained"
elif PRETRAINED == False:
    pretr="nontrained"


# Wrap a Keras model for use with TFF.
def model_fn(m_name, num_labels, seq_len, input_spec):
    model = build_BertNer(m_name, num_labels, seq_len)
    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec,
        loss=MaskedSparseCategoricalCrossentropy()) # reduction=tf.keras.losses.Reduction.NONE))

def eval_model(model, eval_data, do_print=True):
    return evaluate(model, eval_data, 
                    processor.get_label_map(), 
                    processor.token_ind('O'), 
                    processor.token_ind('[SEP]'),
                    processor.token_ind('[PAD]'), 
                    do_print=do_print)
    
eval_data_batched = batch_features(eval_features, processor.get_labels(), SEQ_LEN, tokenizer, batch_size=64)


In [ ]:
num_clients, num_train_clients = 100, 10
assert num_clients >= num_train_clients

dataset_list = split_to_tf_datasets(train_features, num_clients, batch_size=BATCH_SIZE)
trainer = tff.learning.build_federated_averaging_process(
    model_fn=lambda: model_fn(MODEL, processor.label_len(), SEQ_LEN, dataset_list[0].element_spec),
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(5e-3),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(5e-3),
    use_experimental_simulation_loop=True
)

state = trainer.initialize()
if PRETRAINED:
    state = state_from_checkpoint(state, build_BertNer(MODEL, processor.label_len(), SEQ_LEN), MODEL)

res_list=[]
examples = 0
for rnd_ind in range(1, 501):
    train_data = list(np.random.choice(dataset_list, num_train_clients, replace=False))
    state, metrics = trainer.next(state, train_data)
    print("Round", rnd_ind, "Loss:", metrics['train']['loss'], "Examples:", metrics['stat']['num_examples'])
    examples = metrics['stat']['num_examples']
    
    # Ne treba svaku rundu gledati tocnost, moze svakih 10 (jedna epoha kumulativno)
    if rnd_ind % num_train_clients == 0:
        
        model = build_BertNer(MODEL, processor.label_len(), SEQ_LEN)
        state_model = state_to_model(state, model)
        res = eval_model(state_model, eval_data_batched, do_print=True)
        
        res['Round'] = rnd_ind
        res['Examples'] = examples
        res_list.append(res)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

with open("log/results-"+modelSize+"-"+pretr+"-"+datas+".json", "w") as outfile:
    json.dump({'results': res_list, 'model': MODEL, 'seq_len': SEQ_LEN, 
               'pretrained': PRETRAINED, 'batch_size': BATCH_SIZE}, outfile, indent=None, cls=NpEncoder)

Round 1 Loss: 1.7330163 Examples: 1280
Round 2 Loss: 2.5136364 Examples: 1280
Round 3 Loss: 1.8139219 Examples: 1280
Round 4 Loss: 2.393733 Examples: 1280
Round 5 Loss: 1.4863982 Examples: 1280
Round 6 Loss: 1.3764458 Examples: 1280
Round 7 Loss: 1.2955384 Examples: 1280
Round 8 Loss: 1.2382039 Examples: 1280
Round 9 Loss: 1.4503593 Examples: 1280
Round 10 Loss: 1.1993773 Examples: 1280


Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000      1668
        MISC     0.0000    0.0000    0.0000       702
         ORG     0.0000    0.0000    0.0000      1661
         PER     0.0000    0.0000    0.0000      1617

   micro avg     0.0000    0.0000    0.0000      5648
   macro avg     0.0000    0.0000    0.0000      5648
weighted avg     0.0000    0.0000    0.0000      5648

Round 11 Loss: 1.4351972 Examples: 1280
Round 12 Loss: 1.2894019 Examples: 1280
Round 13 Loss: 1.219518 Examples: 1280
Round 14 Loss: 1.140353 Examples: 1280
Round 15 Loss: 1.2642585 Examples: 1280
Round 16 Loss: 1.1855897 Examples: 1280
Round 17 Loss: 1.27582 Examples: 1280
Round 18 Loss: 1.3029983 Examples: 1280
Round 19 Loss: 1.123728 Examples: 1280
Round 20 Loss: 1.1636035 Examples: 1280


Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000      1668
        MISC     0.0000    0.0000    0.0000       702
         ORG     0.0000    0.0000    0.0000      1661
         PER     0.0000    0.0000    0.0000      1617

   micro avg     0.0000    0.0000    0.0000      5648
   macro avg     0.0000    0.0000    0.0000      5648
weighted avg     0.0000    0.0000    0.0000      5648

Round 21 Loss: 1.1541559 Examples: 1280
Round 22 Loss: 1.3145925 Examples: 1280
Round 23 Loss: 1.1620369 Examples: 1280
Round 24 Loss: 1.2610465 Examples: 1280
Round 25 Loss: 1.2099259 Examples: 1280
Round 26 Loss: 1.3500121 Examples: 1280
Round 27 Loss: 1.2684891 Examples: 1280
Round 28 Loss: 1.2069039 Examples: 1280
Round 29 Loss: 1.2091788 Examples: 1280
Round 30 Loss: 1.2826099 Examples: 1280


...

              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000      1668
        MISC     0.0000    0.0000    0.0000       702
         ORG     0.0000    0.0000    0.0000      1661
         PER     0.0000    0.0000    0.0000      1617

   micro avg     0.0000    0.0000    0.0000      5648
   macro avg     0.0000    0.0000    0.0000      5648
weighted avg     0.0000    0.0000    0.0000      5648

Round 121 Loss: 1.3017219 Examples: 1280
Round 122 Loss: 1.2518464 Examples: 1280
Round 123 Loss: 1.0611212 Examples: 1280
Round 124 Loss: 1.286798 Examples: 1280
Round 125 Loss: 1.1985056 Examples: 1280
Round 126 Loss: 1.1270322 Examples: 1280
Round 127 Loss: 1.1755575 Examples: 1280
Round 128 Loss: 1.1350667 Examples: 1280
Round 129 Loss: 1.3634192 Examples: 1280
Round 130 Loss: 1.3674731 Examples: 1280


Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000      1668
        MISC     0.0000    0.0000    0.0000       702
         ORG     0.0000    0.0000    0.0000      1661
         PER     0.0000    0.0000    0.0000      1617

   micro avg     0.0000    0.0000    0.0000      5648
   macro avg     0.0000    0.0000    0.0000      5648
weighted avg     0.0000    0.0000    0.0000      5648

Round 131 Loss: 1.2921798 Examples: 1280
Round 132 Loss: 1.1783385 Examples: 1280
Round 133 Loss: 1.172753 Examples: 1280
Round 134 Loss: 1.2645515 Examples: 1280
Round 135 Loss: 1.277893 Examples: 1280
Round 136 Loss: 1.1472983 Examples: 1280
Round 137 Loss: 1.2720268 Examples: 1280
Round 138 Loss: 1.2242353 Examples: 1280
Round 139 Loss: 1.302057 Examples: 1280
Round 140 Loss: 1.052561 Examples: 1280


Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         LOC     0.0000    0.0000    0.0000      1668
        MISC     0.0000    0.0000    0.0000       702
         ORG     0.0000    0.0000    0.0000      1661
         PER     0.0000    0.0000    0.0000      1617

   micro avg     0.0000    0.0000    0.0000      5648
   macro avg     0.0000    0.0000    0.0000      5648
weighted avg     0.0000    0.0000    0.0000      5648

